## **i. Import the libraries**
---

In [1]:
#!pip install -U matplotlib

In [2]:
#!pip install cmake

In [3]:
#!pip install dlib

In [4]:
import cv2 as cv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import dlib
from PIL import Image
from threading import Thread
import time

# iv. Judge glasses on video
---

In [5]:
font_1=cv.FONT_HERSHEY_SIMPLEX

In [6]:
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [7]:
# create a class for capturing the video frames
# This class handles the video stream 
# captured from the WebCam

class VideoStream:
    def __init__(self, stream):
        self.video = cv.VideoCapture(stream)
        
        # Setting the FPS for the video stream
        self.video.set(cv.CAP_PROP_FPS, 60)

        if self.video.isOpened() is False:
            print("Can't accessing the webcam stream.")
            exit(0)

        self.grabbed , self.frame = self.video.read()
        self.stopped = True
        
        # Creating a thread
        self.thread = Thread(target=self.update)
        self.thread.daemon = True
    
    def start(self):
        self.stopped = False
        self.thread.start()

    def update(self):
        while True :
            if self.stopped is True :
                break
            self.grabbed , self.frame = self.video.read()
        self.video.release()

    def read(self):
        return self.frame

    def stop(self):
        self.stopped = True

In [8]:
# Start capturing video frames through Webcam
# Capturing video through the WebCam. 0 represents the
# default camera. You need to specify another number for
# any external camera

video_stream = VideoStream(stream=0)
video_stream.start()

In [9]:
while True:
    if video_stream.stopped is True:
        break
    else:
        frame = video_stream.read()
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        
        rects = detector(gray, 1)
        
        for i, face_rect in enumerate(rects):
            left = face_rect.left() 
            top = face_rect.top()
            width = face_rect.right() - left
            height = face_rect.bottom() - top

            cv.rectangle(frame, (left, top), (left+width, top+height), (0,255,0), 2)
            cv.putText(frame, f"Face {i+1}", (left - 10, top - 10), font_1, 0.7, (0, 255, 0), 2, cv.LINE_AA)

            frame_crop = frame[top + 10:top+height-100, left + 30: \
            left+width - 20]
#            cv.imshow("Cropped Frame", frame_crop)
            
            img_blur = cv.GaussianBlur(np.array(frame_crop),(3,3),sigmaX=1.7, sigmaY=1.7)
            edges = cv.Canny(image =img_blur, threshold1=100, threshold2=200)
            cv.imshow("Canny Filter", edges)

            edges_center = edges.T[(int(len(edges.T)/2))]
            if 255 in edges_center:
                cv.rectangle(frame, (left, top+height), (left+width,top+height+40), (0,255,0), cv.FILLED)
                cv.putText(frame, "Glass is Present", (left+10,top+height+20), font_1, 0.65, (255, 255, 255), 2, cv.LINE_AA)
            else:
                cv.rectangle(frame, (left, top+height), (left+width,top+height+40), (0,255,0), cv.FILLED)
                cv.putText(frame, "No Glass", (left+10, top+height+20),font_1, 0.65, (0, 0, 255), 2, cv.LINE_AA)

        # delay for processing a frame
        delay = 0.04
        time.sleep(delay)

    cv.imshow("Result", frame)
    
    key = cv.waitKey(1)
    # Press 'q' for stop the executing of the program
    if key == ord('q'):
        break

error: OpenCV(4.6.0) C:\b\abs_74oeeuevib\croots\recipe\opencv-suite_1664548340488\work\modules\imgproc\src\smooth.dispatch.cpp:617: error: (-215:Assertion failed) !_src.empty() in function 'cv::GaussianBlur'


In [ ]:
# stop capturing video frames and close all windows

video_stream.stop()
cv.destroyAllWindows()